In [1]:
## Importing necessary Libraries 
import ctypes, inspect, os, graphlab
from graphlab import SFrame
import pandas as pd

In [2]:
# This code reads the data into a graphlab Sframe. The dataset used is an Office Products dataset containing 53,257 reviews   
# retrieved from: http://jmcauley.ucsd.edu/data/amazon/ citation: "Image-based recommendations on styles and 
# substitutes" J. McAuley, C. Targett, J. Shi, A. van den Hengel, SIGIR, 2015
Office_Products = graphlab.SFrame.read_json('Office_Products_5.json', orient='lines')

2017-04-19 06:55:42,257 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: C:\Users\Chauncey\AppData\Local\Temp\graphlab_server_1492599339.log.0


Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 100 lines in 6.13201 secs.

This non-commercial license of GraphLab Create for academic use is assigned to raggiechauncey@gmail.com and will expire on April 23, 2017.
------------------------------------------------------

Read 50429 lines. Lines per second: 39896.1

Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 53258 lines in 1.28401 secs.


Inferred types from first line of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
## Exploring the dataset
Office_Products.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU,ARH
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO,"Let it Be ""Alan"""
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5,Mark B
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5,R. D Johnson
B00000JBLH,"[0, 0]",5.0,For simple calculationsand discounted cash ...,"08 4, 2013",A2XRMQA6PJ5ZJ8,Roger J. Buffington
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE,scott_from_dallas
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8,W. B. Halper
B00000JBLH,"[0, 0]",5.0,Bought this for my bossbecause he lost his. He ...,"11 14, 2013",AA8M6331NI1EN,ZombieMom
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ,Comdet
B00000JBLU,"[0, 0]",5.0,"I love this calculator,big numbers and calcu ...","12 2, 2013",A1RTVWTWZSIC94,"Hb ""Black Beauty"""


In [4]:
## Counting the number of rows 
len(Office_Products)

53258

In [5]:
## Setting target as the notebook
graphlab.canvas.set_target('browser')

In [6]:
## Checking the rating distribution. Most reviews have a 5 rating. 30,327 votes (56.944%)
Office_Products['overall'].show(view='Categorical') 

Canvas is accessible via web browser at the URL: http://localhost:59903/index.html
Opening Canvas in default web browser.


In [7]:
# building a word count vector for each review 
Office_Products['word_count'] = graphlab.text_analytics.count_words(Office_Products['reviewText'])

In [12]:
# Removing all stopwords
Office_Products['No_Stop_words'] = Office_Products['word_count'].dict_trim_by_keys(graphlab.text_analytics.stopwords(), exclude=True)
#Removing all punctuation
Office_Products['No_punct'] = Office_Products['No_Stop_words'].dict_trim_by_keys(['!','.',':','*','~','(2),'], exclude=True)
#Calculating TF-IDF
Office_Products['tfidf'] = graphlab.text_analytics.tf_idf(Office_Products['No_punct'])

In [13]:
# Limiting the review data to only helpful reviews. The ‘helpful’ column contains values like ‘[x, y]’.
# The first value represents the number of helpful votes, the second represents overall votes. 
# If at least 60% consider the review helpful, we want those reviews. A binary 1 or 0 column will identify the review as helpful or not.
import numpy as np
Office_Products['helpful_count'] = Office_Products['helpful'].apply(lambda x: x[0])
Office_Products['overall_count'] = Office_Products['helpful'].apply(lambda x: x[1])
Office_Products['helpful_percent'] = Office_Products['helpful_count'] / Office_Products['overall_count']
Office_Products['helpful_review'] = Office_Products['helpful_percent'] >= .6
# Removing reviews with no votes 
Office_Products = Office_Products[Office_Products['overall_count']!= 0 ]

In [14]:
Office_Products.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ
B00000JBLU,"[2, 2]",5.0,I've had mine for 4 yearsnow and use it almost ...,"07 10, 2007",A30U2QQN2FFHE9
B00000JBLU,"[1, 2]",5.0,"If you need a calculatorthat prints, this is not ...","07 15, 2008",A2W0XCWOK3MW6F
B00000JBLU,"[6, 6]",5.0,"Texas Instruments makesan excellent, relatively ...","06 22, 2010",A2R6RA8FRBS608


In [15]:
# Building the sentiment classifier. Performing a random 80/20 split to select the training and testing data.
train_data, test_data = Office_Products.random_split(.8, seed=0)

In [16]:
sentiment_model_Logist = graphlab.logistic_classifier.create(train_data,
                                                            target = 'helpful_review',
                                                            features = ['tfidf'],
                                                            validation_set = test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 16902

Number of classes           : 2

...

| 6         | 14       | 1.000000  | 3.487067     | 0.984736          | 0.664242            |

| 10        | 19       | 1.000000  | 4.642091     | 0.993196          | 0.687515            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [17]:
## evaluating the model
sentiment_model_Logist.evaluate(test_data, metric = 'roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+------+------+
 | threshold |      fpr       |      tpr       |  p   |  n   |
 +-----------+----------------+----------------+------+------+
 |    0.0    |      1.0       |      1.0       | 2938 | 1187 |
 |   1e-05   | 0.996630160067 | 0.999659632403 | 2938 | 1187 |
 |   2e-05   | 0.995787700084 | 0.999659632403 | 2938 | 1187 |
 |   3e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   4e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   5e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   6e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   7e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   8e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   9e-05   | 0.994945240101 | 0.998978897209 | 2938 | 1187 |
 +-----------+----------------+----------------+------+------+
 [100001 row

In [18]:
sentiment_model_Logist.show(view = 'Evaluation')

Canvas is accessible via web browser at the URL: http://localhost:59903/index.html
Opening Canvas in default web browser.


In [19]:
## Finding the ID of the most frequently reviewed item.  Value	Count	Percent
# B000WU4H5C 156 reviews (0.742%)
Office_Products['asin'].show()

Canvas is updated and available in a tab in the default browser.


In [20]:
B000WU4H5C_Reviews = Office_Products[Office_Products['asin'] == 'B000WU4H5C']

In [21]:
len(B000WU4H5C_Reviews)

156

In [22]:
# Applying the Logistic Regression model to B000WU4H5C_Reviews
B000WU4H5C_Reviews['Predicted_Helpfulness_LR'] = sentiment_model_Logist.predict(B000WU4H5C_Reviews, output_type = 'probability')

In [ ]:
B000WU4H5C_Reviews.head()

In [23]:
## Sorting reviews based on the predicted sentiment 
B000WU4H5C_Reviews = B000WU4H5C_Reviews.sort('Predicted_Helpfulness_LR', ascending =False)

In [24]:
B000WU4H5C_Reviews.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID
B000WU4H5C,"[1, 1]",4.0,"This is good for work;it's basic black, ...","03 30, 2011",A1IU7S4HCK1XK0
B000WU4H5C,"[25, 28]",4.0,"Years ago, I had akeyboard wrist rest that ...","04 5, 2011",A3JCWGHCVC2ZNF
B000WU4H5C,"[5, 6]",5.0,"I've used several wristrests over the years, ...","04 19, 2011",A2JAEZ0FMAMJVW
B000WU4H5C,"[3, 5]",4.0,The 3M gel wrist rest Irecieved as part of my ...,"03 31, 2011",AAEHOTZ1GRAUX
B000WU4H5C,"[4, 6]",5.0,I don't use wrist restsvery often because they ...,"10 5, 2010",A2KFWHHUFVKFXZ
B000WU4H5C,"[7, 11]",1.0,At first I loved this. Itfelt reasonably ...,"06 8, 2011",A9AGFKGEQH5DE
B000WU4H5C,"[3, 4]",5.0,I have lived without thiswrist rest all my life ...,"04 18, 2011",A2068BC3ZXAVJQ
B000WU4H5C,"[0, 2]",4.0,"My job forces me to usethe mouse, almost non- ...","04 17, 2011",ATS7LYGLUX34L
B000WU4H5C,"[0, 1]",5.0,I bought a cheaper gelversion and within 6 ...,"05 21, 2014",A2QD4VQQGZDLBP
B000WU4H5C,"[0, 1]",5.0,"This gel pad is prettycool, it keeps your w ...","03 24, 2011",A3NZVCKY68MUFJ


In [25]:
B000WU4H5C_Reviews[0]['reviewText']

"This is good for work; it's basic black, leatherette and antibacterial. I notice our keyboards get grubby. So this is good to have a leatherette that can be wiped down now and then, rather than a stretch knit cover, which is how some gel supports are covered. I prefer this leatherette for cleaning, though I like the feel of gel-under-fabric better for comfort.The wrist pad has a membrane you peel off the first time you use it. The underside is sticky but not gummy--it stays put on your desk and doesn't slide around. The leatherette is easy to wash or give it a quick squirt of even hand sanitizer. The gel is nice and supportive under the wrists. It is a bit firmer than others I've used, but I actually like it better than my last wrist pad. I typically use a wrist-rest mouse pad such as the3M Large Mouse Pad with Gel Wrist Restthat is similar to the leatherette of this keyboard rest.*UPDATE: I've had this for over a year and a half, and it remains my favorite wrist rest (if I don't use 

In [26]:
B000WU4H5C_Reviews[-1]['reviewText']

'I revisited this review after using the product for 2 weeks.  My original review appears below.  I had noted that I use wrist rests because of arthritis, and I have used them for several years and tried several brands.  While the 3m leatherette pad may look nice on a desk it is not as comfortabel as the fellows pad and does not do as well in terms of reducing fatige, or joint pain.  I have substantially more joint pain after using this pad for 2 weeks than I did with the fellows gel pad .Fellowes Purple Gel Crystal Flex Rest Transparent Mini Wrist Restwhich is a fraction the cost.  Which is better for you will depend on your priorities and needs.  The 3m pad is clearly a nice. more professional looking pad, but for those of us with arthritis (I am assuming other arthritis sufferers will feel this way, though I could be wrong) comfort, reduced fatigue and joint pain are more of an issue, at least for myself I can say this is the case. So, after trying it for 2 weeks, I am going back to

In [ ]:
B000WU4H5C_Reviews[0]['summary']

In [ ]:
B000WU4H5C_Reviews[-1]['summary']

In [27]:
## Trying a SVM model
sentiment_model_SVM = graphlab.svm_classifier.create(train_data,
                                                     target = 'helpful_review',
                                                     features = ['tfidf'],
                                                     validation_set = test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

SVM:

--------------------------------------------------------

Number of examples          : 16902

Number of classes           : 2

...

| 6         | 14       | 1.000000  | 2.375564     | 0.976748          | 0.700848            |

| 10        | 18       | 1.000000  | 3.363082     | 0.990001          | 0.702303            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [28]:
## Evaluating the model
coefficients = sentiment_model_SVM['coefficients']

In [29]:
# Make predictions (as margins, or class)
predictions = sentiment_model_SVM.predict(Office_Products)    # Predicts 0/1
predictions = sentiment_model_SVM.predict(Office_Products, output_type='margin')

In [30]:
results = sentiment_model_SVM.evaluate(Office_Products)

In [31]:
results

{'accuracy': 0.9335616112617111, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  244  |
 |      0       |        1        |  1153 |
 |      0       |        0        |  4842 |
 |      1       |        1        | 14788 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns], 'f1_score': 0.954896199916056, 'precision': 0.9276707860234615, 'recall': 0.9837679616817456}

In [32]:
# Applying the SVM model to B000WU4H5C_Reviews. Note: SVM does not currently support predictions as probability estimates.
B000WU4H5C_Reviews['Predicted_Helpfulness_SVM'] = sentiment_model_SVM.predict(B000WU4H5C_Reviews, output_type = 'class')

In [33]:
B000WU4H5C_Reviews.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID
B000WU4H5C,"[1, 1]",4.0,"This is good for work;it's basic black, ...","03 30, 2011",A1IU7S4HCK1XK0
B000WU4H5C,"[25, 28]",4.0,"Years ago, I had akeyboard wrist rest that ...","04 5, 2011",A3JCWGHCVC2ZNF
B000WU4H5C,"[5, 6]",5.0,"I've used several wristrests over the years, ...","04 19, 2011",A2JAEZ0FMAMJVW
B000WU4H5C,"[3, 5]",4.0,The 3M gel wrist rest Irecieved as part of my ...,"03 31, 2011",AAEHOTZ1GRAUX
B000WU4H5C,"[4, 6]",5.0,I don't use wrist restsvery often because they ...,"10 5, 2010",A2KFWHHUFVKFXZ
B000WU4H5C,"[7, 11]",1.0,At first I loved this. Itfelt reasonably ...,"06 8, 2011",A9AGFKGEQH5DE
B000WU4H5C,"[3, 4]",5.0,I have lived without thiswrist rest all my life ...,"04 18, 2011",A2068BC3ZXAVJQ
B000WU4H5C,"[0, 2]",4.0,"My job forces me to usethe mouse, almost non- ...","04 17, 2011",ATS7LYGLUX34L
B000WU4H5C,"[0, 1]",5.0,I bought a cheaper gelversion and within 6 ...,"05 21, 2014",A2QD4VQQGZDLBP
B000WU4H5C,"[0, 1]",5.0,"This gel pad is prettycool, it keeps your w ...","03 24, 2011",A3NZVCKY68MUFJ


In [34]:
## Sorting reviews based on the predicted sentiment 
B000WU4H5C_Reviews = B000WU4H5C_Reviews.sort('Predicted_Helpfulness_SVM', ascending =False)

In [35]:
B000WU4H5C_Reviews[0]['reviewText']

"Years ago, I had a keyboard wrist rest that was comfortable but it had some major downsides:  a) It was covered in a nylon-type fabric which got stained fairly quickly and was not easy to clean; and b) it was thick, wide and rectangular and it was forever falling off my keyboard tray.  I eventually threw it away not because it fell apart or stopped working, but because it LOOKED disgusting and was a nuisance to keep on the keyboard tray.This 3M gel wrist rest improves on both of those weaknesses:  1) It is leatherette vs. fabric, so any spills can easily be wiped up; and 2) while, like my old one, this rest runs the full length of my keyboard, it is tapered at the ends so that it fits my keyboard tray better.  I haven't felt it MOVE much less fall off the keyboard tray.  The rest comes with a film across the bottom surface.  Once I peeled off the film and placed the rest on my keyboard tray, it has stayed in place.  It's not GLUED there--i.e., you can pick it up and move it and it's n

In [36]:
B000WU4H5C_Reviews[-1]['reviewText']

"This arrived today and I love it. It is a firm pad which supports my mouse-wrist perfectly. I didn't realize how much pressure I was using on my wrist but this is exactly what I needed. Other pads I tried were too soft and only made a cushion, while this one is an actual support that alleviates bone pain and shoulder stress."

In [41]:
B000WU4H5C_Reviews.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID
B000WU4H5C,"[25, 28]",4.0,"Years ago, I had akeyboard wrist rest that ...","04 5, 2011",A3JCWGHCVC2ZNF
B000WU4H5C,"[0, 1]",2.0,Before I mention thethings that could be ...,"05 6, 2011",A10PEXB6XAQ5XF
B000WU4H5C,"[1, 3]",4.0,"What can one say, itssimple, black, ...","10 21, 2010",AONUCJBC85BL
B000WU4H5C,"[0, 2]",5.0,3M's Gel Wrist Rest withantimicrobial protection ...,"03 29, 2011",A2RKI7RPP5CJMG
B000WU4H5C,"[0, 2]",5.0,This wrist rest isperfect for our mouse ...,"04 21, 2011",A2JE8OS1NN9YDB
B000WU4H5C,"[0, 1]",5.0,I almost gave up on usinggel wrests for my tired ...,"04 4, 2011",AEASNN4Z1IWB2
B000WU4H5C,"[1, 1]",4.0,"This is good for work;it's basic black, ...","03 30, 2011",A1IU7S4HCK1XK0
B000WU4H5C,"[0, 1]",5.0,I bought a cheaper gelversion and within 6 ...,"05 21, 2014",A2QD4VQQGZDLBP
B000WU4H5C,"[0, 2]",4.0,"My job forces me to usethe mouse, almost non- ...","04 17, 2011",ATS7LYGLUX34L
B000WU4H5C,"[3, 4]",5.0,I have lived without thiswrist rest all my life ...,"04 18, 2011",A2068BC3ZXAVJQ


In [38]:
B000WU4H5C_Reviews[1]['reviewText']

'Before I mention the things that could be improved upon, here are the things I like about this wrist rest: it\'s about the right height for my keyboard, it has about the right "firmness," it doesn\'t get hot after having your hands / wrist on it for an extended period of time, and the drop of coffee I spilled on it wiped right up with no staining.All of this could be improved if the manufacturer did just a few things.  First, it is way too long as I prefer to have these things about the same length as my keyboard.  They are trying to accomodate a mouse in addition to the keyboard which is a nice thing, but I\'m not one of those users who has the mouse immediately next to my keyboard - it is several inches away and, as a result, this extra length just gets in the way of everything else that is piled on my desk.My main complaint about this wrist rest, however, is it is not wide enough.  I don\'t think I have abnormally large hands (I don\'t), but pulling out my ruler I have the edge of 

In [39]:
B000WU4H5C_Reviews[2]['reviewText']

'What can one say, its simple, black, comfortable, fits nicely on my desk. It also has some nice "grippy" surface that keeps it in place. I use it every day.'

In [40]:
B000WU4H5C_Reviews[3]['reviewText']

"3M's Gel Wrist Rest with antimicrobial protection is wonderful.  It sits on my desk and stays put with a simple bottom that adheres to the surface without leaving a residue.  It features a soft, leather-like covering and the padding is both comfortable and supportive.My wrist sits comfortably on the rest and doesn't stick to it.  This allows for free movement of my hand as I click around on my computer.I highly recommend the 3M Gel Wrist Rest for both home and office."

In [ ]:
Office_Products['tfidf'] = graphlab.text_analytics.tf_idf(Office_Products['No_Stop_words'])


In [ ]:
Office_Products.head()

In [37]:
Office_Products[['tfidf']].stack('tfidf', new_column_name=['word', 'tfidf']).sort('tfidf', ascending =False)

word,tfidf
keyboard,335.512802873
keyboard,299.987917863
mouse,209.726857377
laptop,207.069275722
tray,199.590934815
tray,193.720613203
pro-100,190.018591482
caveman,185.208308492
pens2,179.164126893
2270dw,175.947893068
